In [2]:
%matplotlib inline

from dit.divergences import jensen_shannon_divergence
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from dateutil.parser import parse
from haversine import haversine
import plotly.graph_objs as go
from shapely.geometry import *
from datetime import datetime  
import plotly.plotly as py
import scipy.stats as st
import geopandas as gpd
from scipy import stats
import pandas as pd
import numpy as np  
import operator
import geojson
import math 
import sys 
import dit
import numpy
import pandas

# data.london.gov.uk/dataset/statistical-gis-boundary-files-london 
singapore = gpd.read_file('../data/MP14_SUBZONE_WEB_PL.shp')
# from the prj file: http://www.prj2epsg.org/epsg/27700
singapore = singapore.to_crs({'init': 'epsg:4277'})   

# path to data
venues = "../../../raw_data/venues/London_venues.txt"
transitions = "../../../raw_data/transitions/London_transitions.txt"

# venue ID -> name
venue_id_to_name = {} 
# venue ID -> coordinates for that venue
venue_id_to_coords = {} 
# venue ID -> SPECIFIC category
venue_id_to_category_specific = {} 
# venue ID -> GENERAL category
venue_id_to_category_general = {} 
# venue ID -> opening date
venue_id_to_opening = {}  

### get all venues 
with open(venues, 'r') as f:
    for rows in f:
        rows = rows.split("\t")
        # unique ID 
        ID = rows[0] 
        # name
        name = rows[1]
        venue_id_to_name[ID] = name 
        # coordinates 
        coordinates = (float(rows[2]), float(rows[3]))
        venue_id_to_coords[ID] = coordinates  
        # specific category
        type_venue = rows[7]
        venue_id_to_category_specific[ID] = type_venue  
        # general categorgy
        type_venue = rows[8]
        venue_id_to_category_general[ID] = type_venue  
        
        # date
        date = rows[9].strip()  
        date_cleaned = datetime.strptime(date, '%Y-%m-%d')
        venue_id_to_opening[ID] = date_cleaned
        
LIST_OF_VENUES = venue_id_to_opening.keys()

# venue ID -> list of *FIRST* checkins to that venue       
venue_id_to_times = {} 
with open(transitions, 'r') as f:
    for rows in f:
        rows = rows.split("\t")
        location1 = rows[0]
        time1 = rows[1] 
        if location1 in venue_id_to_coords:
            venue_id_to_times.setdefault(location1, [])
            time_cleaned = datetime.fromtimestamp(int(time1))
            venue_id_to_times[location1].append(time_cleaned)
             
# venue id to the number of checkins to that venue 
venue_id_to_num = {}
for v in LIST_OF_VENUES:
    if v not in venue_id_to_times:
        continue 
    venue_id_to_num[v] = len(venue_id_to_times[v])    

IOError: no such file or directory: '../data/MP14_SUBZONE_WEB_PL.shp'

In [ ]:
places_opened_to_num = {}
for v in venue_id_to_opening: 
    if v not in venue_id_to_num: 
        continue 
    date = venue_id_to_opening[v]
    year = date.year
    month = date.month 
    places_opened_to_num[v] = venue_id_to_num[v]
         
# list of new venues that have at least 100 checkins 
venues_above_threshold = [venue for venue in places_opened_to_num if places_opened_to_num[venue] > 100]
print len(venues_above_threshold)

In [3]:
 
# This takes a long time. It gives us info about which venues are near other venues. 
"""
from haversine import haversine
print "GOAL: ", len(venues_above_threshold)
i = 0
venue_to_set = {}
for v in venues_above_threshold: 
    i += 1
    venue_to_set.setdefault(v, set())
    c1 = venue_id_to_coords[v]
    for v2 in venue_id_to_coords: 
        if v2 == v: 
            continue
        c2 = venue_id_to_coords[v2]
        distance = haversine(c1, c2)
        if distance < .5:  
            venue_to_set[v].add(v2) 
    if i % 1000 == 0: 
        print i
   
 
import cPickle

with open("500m_nearby_venues_pickle_london.txt", "w") as fp:
    cPickle.dump(venue_to_set, fp)
""" 
with open("pickledump.txt", "r") as fp:
    venue_to_set = cPickle.load(fp)

IOError: [Errno 2] No such file or directory: 'pickledump.txt'

In [4]:
# check if there's data last 6 months
# is there less than 12 checkins after july 2013
venues_closed = set()
# whether venue is open or closed 
venue_to_status = {}
for v in venues_above_threshold:
    if v not in venue_id_to_times: 
        continue
    times = venue_id_to_times[v]
    checkins_after = 0
    for date in times:
        year = date.year
        month = date.month
        if year == 2013 and month > 6 or year > 2014:
            checkins_after += 1 
    if checkins_after <= 6: # less than one checkin a month
        venues_closed.add(v)     
        venue_to_status[v] = 0
    else: 
        venue_to_status[v] = 1 

In [5]:
venue_to_surrdng_area = venue_to_set

# when people end up at this ward, from where do they come?
# that same ward or a different ward?
venue_inflow_transitions = {}
venue_outflow_transitions = {}
#venue_within_transitions = {}

venue_distance_entering = {}
venue_distance_leaving = {}
venue_speed_entering = {}
venue_speed_leaving = {}
  
error_counter = 0
i = 0
with open(transitions, 'r') as f:
    for rows in f: 
        i += 1 
        if i % 1000000 == 0:
            print i
        rows = rows.split("\t")
        location_1 = rows[0]
        time1 = rows[1]
        location_2 = rows[2]
        time2 = rows[3]  
 
        if location_1 not in venue_to_surrdng_area or location_2 not in venue_to_surrdng_area: 
            error_counter += 1
            continue
        if location_1 not in venue_id_to_coords or location_2 not in venue_id_to_coords: 
            continue

        coords_1 = venue_id_to_coords[location_1]
        coords_2 = venue_id_to_coords[location_2]
        distance_current = haversine(coords_1, coords_2)

        timeTravelled = int(time2) - int(time1)
        timeTravelled = (timeTravelled / 60.0)/ 60.0
        if timeTravelled > 0:
            speed_current = distance_current / timeTravelled
        else: 
            speed_current = 0

        #
        venue_outflow_transitions.setdefault(location_1, 0)
        venue_outflow_transitions[location_1] += 1 

        venue_speed_leaving.setdefault(location_1, [])
        venue_speed_leaving[location_1].append(speed_current)
        venue_distance_leaving.setdefault(location_1, [])
        venue_distance_leaving[location_1].append(distance_current)

        #
        venue_inflow_transitions.setdefault(location_2, 0)
        venue_inflow_transitions[location_2] += 1 

        venue_speed_entering.setdefault(location_2, [])
        venue_speed_entering[location_2].append(speed_current)
        venue_distance_entering.setdefault(location_2, [])
        venue_distance_entering[location_2].append(distance_current)

        #if location_1 in v and location_2 in v:
            #srrndng_to_num_within_transitions.setdefault(k, 0)
            #srrndng_to_num_within_transitions[k] += 1  

1000000
2000000
3000000
4000000


In [6]:
# calculate entropy
srrdng_to_entropy = {}
for current_v in venue_to_surrdng_area:  
    dict_cat_to_count = {}  
    
    list_venues = venue_to_surrdng_area[current_v] 
    for surrounding_v in list_venues: 
        categ = venue_id_to_category_general[surrounding_v]
        dict_cat_to_count.setdefault(categ, 0)
        dict_cat_to_count[categ] += 1
        
    sum_values = sum(dict_cat_to_count.values())
    perc_dict = {}
    for raw_vals in dict_cat_to_count: 
        perc_dict[raw_vals] = dict_cat_to_count[raw_vals] / float(sum_values)
    
    sum_vals = 0
    for k, perc in perc_dict.items():  
        sum_vals += perc * math.log(perc) 
    sum_vals *= -1
    if len(perc_dict) != 0:
        entropy = sum_vals / len(perc_dict)
    else:
        entropy = 0
    srrdng_to_entropy[current_v] = entropy 

In [ ]:
"""
import csv
venue_to_kas_status = {}
with open("london-venues-wlabel.csv", "rb") as f:
    reader = csv.reader(f)
    for row in reader:
        venue_to_kas_status[row[0]] = row[6] 
"""

In [7]:
all_general = list(set(venue_id_to_category_general.values()))
all_specific = list(set(venue_id_to_category_specific.values()))

myData = np.array([["venue", "in", "out", \
                    "ratio in/out", "distance","speed","dis_std", "spe_std", "dis_lv", \
                    "spe_lv", "dis_lv_std", "spe_lv_std", "spec", "gen", \
                    "spec_count", "gen_count", "all_count", "spec_gen", "gen_all", "entropy","closed or open"]])
NUM_FEATURES = 20
count = 0

for new_venue in venues_above_threshold: 
    # travel out of city bounds 
    
    if new_venue not in venue_inflow_transitions or new_venue not in venue_outflow_transitions: 
        continue 
        
    inner = np.array([]) 
    surrounding = venue_to_surrdng_area[new_venue]
    
    inflow_sum = 0
    outflow_sum = 0 
    distance_enter = []
    distance_leave = []
    speed_enter = []
    speed_leave = []
    
    same_spec_cat = 0
    sam_gen_cat = 0
    all_venues = 0 
    
    #if new_venue not in venue_to_kas_status: 
        #continue
    #status = venue_to_kas_status[new_venue]
    status = venue_to_status[new_venue]
    spec = venue_id_to_category_specific[new_venue]
    gen = venue_id_to_category_general[new_venue]
    
    for surround_v in surrounding: 
        if surround_v in venue_inflow_transitions: 
            inflow_sum += venue_inflow_transitions[surround_v]
        if surround_v in venue_outflow_transitions: 
            outflow_sum += venue_outflow_transitions[surround_v]
        if surround_v in venue_distance_entering: 
            distance_enter.append(np.mean(venue_distance_entering[surround_v]))
        if surround_v in venue_distance_leaving: 
            distance_leave.append(np.mean(venue_distance_leaving[surround_v]))
        if surround_v in venue_speed_entering: 
            speed_enter.append(np.mean(venue_speed_entering[surround_v]))
        if surround_v in venue_speed_leaving: 
            speed_leave.append(np.mean(venue_speed_leaving[surround_v]))
        
        spec_surr = venue_id_to_category_specific[surround_v]
        gen_surr = venue_id_to_category_general[surround_v]
        if spec_surr == same_spec_cat: 
            same_spec_cat += 1
        if sam_gen_cat == gen_surr:
            sam_gen_cat += 1
        all_venues += 1
       
    dis_ent_mean = np.mean(distance_enter)
    dis_ent_std = np.std(distance_enter)
    dis_lea_mean = np.mean(distance_leave)
    dis_lea_std =  np.std(distance_leave)
    speed_ent_mean = np.mean(speed_enter)
    speed_ent_std =  np.std(speed_enter)
    speed_lea_mean = np.mean(speed_leave)
    speed_lea_std =  np.std(speed_leave) 
     
    entropy = srrdng_to_entropy[new_venue]
    
    index_spec = all_specific.index(spec)
    index_gen = all_general.index(gen) 
    
    inner = np.append(inner, count) 
    #inner = np.append(inner, within)
    inner = np.append(inner, inflow_sum)
    inner = np.append(inner, outflow_sum)    
    #inner = np.append(inner, float(within) / inflow)
    #inner = np.append(inner, float(within) / outflow)
    if outflow_sum > 0: 
        inner = np.append(inner, float(inflow_sum) / outflow_sum)
    else: 
        inner = np.append(inner, 0)
    inner = np.append(inner, dis_ent_mean)  
    inner = np.append(inner, dis_ent_std)
    inner = np.append(inner, dis_lea_mean)
    inner = np.append(inner, dis_lea_std)
    inner = np.append(inner, speed_ent_mean)
    inner = np.append(inner, speed_ent_std)
    inner = np.append(inner, speed_lea_mean)
    inner = np.append(inner, speed_lea_std)
    
    inner = np.append(inner, same_spec_cat)
    inner = np.append(inner, sam_gen_cat)
    inner = np.append(inner, all_venues)
    if sam_gen_cat != 0:
        inner = np.append(inner, float(same_spec_cat) / sam_gen_cat)
    else:
        inner = np.append(inner, 0)
    if all_venues != 0:
        inner = np.append(inner, float(sam_gen_cat) / all_venues)
    else:
        inner = np.append(inner, 0)
        
    inner = np.append(inner, entropy)
    
    inner = np.append(inner, index_spec)
    inner = np.append(inner, index_gen)
    
    # output
    inner = np.append(inner, status)  
    myData = np.concatenate([myData, [inner]])   
    count += 1

/usr/local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning:

Degrees of freedom <= 0 for slice



In [ ]:
# remove nan
# do as % change
# need to downsample

In [8]:
import csv
myFile = open('features_updated_london.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(myData)  
print("Writing complete")

Writing complete
